In [11]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute

/Users/Daniel/anaconda3/envs/carnegie3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Create simulation datasets

In [5]:
def create_sim_dataset(validation, n_samples, n_rows,  n_sets):
    
    test = pd.read_csv("../data/test_kor.csv")
    frame = ~test.isna()
    
    validation = validation.reindex(frame.columns, axis=1)
    
    result_lst = []
    
    for _ in range(n_sets):

        validation_sampled = validation.sample(n=n_samples, replace=False)

        n_split = n_samples / n_rows

        splited_indexes = np.array_split(validation_sampled.index, n_split)

        problem_df_lst = []
        answer_df_lst = []
        for idx in splited_indexes:
            selected_df = validation.loc[idx]
            selected_df.reset_index(drop=True, inplace=True)
            problem_df_lst.append(selected_df[frame])
            answer_df_lst.append(selected_df[~frame])

        problem_df = pd.concat(problem_df_lst)
        answer_df = pd.concat(answer_df_lst)
        result_lst.append({
            'problem': problem_df, 
            'answer': answer_df
                           
        })
    return result_lst
        

## Create metric for scoring


In [7]:
def calculate_numerical_score(pred_df, true_df, B,  sj=1):
    diff = np.concatenate(pred_df[numeric_predictor].values - true_df[numeric_predictor].values)
    diff = diff[~np.isnan(diff)]
    return B * np.sum(np.exp(-((diff/sj) ** 2)))

def calculate_categorical_score(pred_df, true_df, C):
    return C * np.sum(pred[categorical_predictor].values == pred[categorical_predictor].values)

def calculate_total_score(pred_df, true_df, sj, B, C):
    
    return calculate_numerical_score(pred_df, true_df, B=B, sj=sj) + calculate_categorical_score(pred_df, true_df, C=C)

## Download file from S3

In [9]:
def download_files(filenames):
    
    import boto3
    import botocore
    
    s3 = boto3.resource('s3')
    
    BUCKET_NAME = 'kaj011'
    
    for filename in filenames:
        KEY = 'samsung_challenge/%s' % filename
        
        
        try:
            s3.Bucket(BUCKET_NAME).download_file(KEY, '../data/%s' % filename)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
    
    